In [1]:
import pandas as pd
import numpy as np

import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import string

from tashaphyne.stemming import ArabicLightStemmer
import arabicstopwords.arabicstopwords as stp
from ar_corrector.corrector import Corrector
import re

import nltk
from nltk.corpus import stopwords
import wordcloud
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import DBSCAN, SpectralClustering, MeanShift
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('new.csv')
df2=pd.read_csv('try.csv')

In [3]:
pd.set_option('display.max_columns', 30)

In [4]:
r = [df, df2]
df = pd.concat(r)
df.drop_duplicates(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
dtypes: object(13)
memory usage: 38.4+ KB


In [6]:
df.duplicated().sum()

0

In [7]:
df_Train, df_test = train_test_split(df, test_size=0.2, random_state= 30)

df_train, df_val = train_test_split(df_Train, test_size=0.2, random_state= 30)

In [8]:
print(f'Shape of train:', df_train.shape)
print(f'Shape of validation:', df_val.shape)
print(f'Shape of test:', df_test.shape)

Shape of train: (224, 13)
Shape of validation: (56, 13)
Shape of test: (71, 13)


In [9]:
df_train.isnull().sum()

links              0
court              0
court_id           0
court_date         0
appeal_court      26
appeal_id         26
appeal_date       45
region            45
city              19
judgment_text      0
appeal_text       45
appeal_data      205
appleal_text     205
dtype: int64

In [10]:
df_val.isnull().sum()

links             0
court             0
court_id          0
court_date        0
appeal_court      4
appeal_id         4
appeal_date      12
region           12
city              8
judgment_text     0
appeal_text      12
appeal_data      48
appleal_text     48
dtype: int64

In [11]:
df_train.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    96
 المحكمة التجارية                     87
 محكمة الاستئناف: محكمة الاستئناف      8
 محكمة الاستئناف                       7
 Name: appeal_court, dtype: int64,)

In [12]:
df_val.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    30
 المحكمة التجارية                     21
 محكمة الاستئناف                       1
 Name: appeal_court, dtype: int64,)

In [13]:
df_train['appeal_court'].fillna(df_train['appeal_court'].mode()[0], inplace = True)

In [14]:
df_val['appeal_court'].fillna(df_val['appeal_court'].mode()[0], inplace = True)

In [15]:
df_train.appeal_id.value_counts()

 ٦٥٤                2
 ٦٣٧                2
 ٦٨٦                2
 ٢٦٣٠               2
 ٧٦                 2
                   ..
رقم القرار: ٥٦٣٦    1
رقم القرار: ٥٥٥٠    1
رقم القرار: ٧٦      1
رقم القرار: ٥٢٥٥    1
رقم القرار: ٤٧٢٠    1
Name: appeal_id, Length: 192, dtype: int64

In [16]:
df_val.appeal_id.value_counts()

رقم القرار: ٥٤٧٠    1
رقم القرار: ٥١٣٣    1
 ٤٤٢١               1
رقم القرار: ٥٣٤٣    1
رقم القرار: ٢       1
رقم القرار: ٢٨٩٣    1
 ٤٧٩٥               1
رقم القرار: ٥٢٥٣    1
رقم القرار: ٥٢٦٧    1
رقم القرار: ٥٢٨١    1
 ٥٤٩١               1
رقم القرار: ٢١٥٧    1
 ٥٢٦٧               1
رقم القرار: ٥٣٠٠    1
 ٧٣                 1
 ٥٥٨١               1
 ٥٦٣٦               1
رقم القرار: ٣٤٥٤    1
رقم القرار: ٩٣      1
 ٩٦                 1
 ٢٢                 1
 ٥٤٢                1
رقم القرار: ٩٦      1
رقم القرار: ٣٢٤٧    1
 ٢٨٩٣               1
رقم القرار: ٧٩٥     1
 ٧٥٧                1
رقم القرار: ٧٣      1
رقم القرار: ٣٢٣     1
 ٣٧٥                1
 ٤٤٣٦               1
رقم القرار: ٥٥٠٨    1
 ١٤٥                1
رقم القرار: ٤١٧١    1
رقم القرار: ٣٧٥     1
رقم القرار: ٥٩٨     1
رقم القرار: ٢٣٠٧    1
 ٦٨٨                1
رقم القرار: ٦٨٨     1
رقم القرار: ٥٤٩١    1
 ٣٦                 1
رقم القرار: ٥٤٩٢    1
 ٥٩١                1
رقم القرار: ٤٧١٦    1
رقم القرار: ٥٤٠٣    1
 ٧٠٩      

In [17]:
df_train.dropna(subset=['appeal_id'], inplace=True);

In [18]:
df_val.dropna(subset=['appeal_id'], inplace=True);

In [19]:
df_train['appeal_date'] = df_train['appeal_date'].str.replace('تاريخه:','')

In [20]:
df_val['appeal_date'] = df_val['appeal_date'].str.replace('تاريخه:','')

In [21]:
df_train['appeal_id'] = df_train['appeal_id'].str.replace('رقم القرار:','')

In [22]:
df_val['appeal_id'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [23]:
df_val['appeal_date'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [24]:
df_train['court'] = df_train['court'].str.replace('المحكمة:','')

In [25]:
df_train['court_id'] = df_train['court_id'].str.replace('رقم القضية - القرار:','')

In [26]:
df_val['court_id'] = df_val['court_id'].str.replace('رقم القضية - القرار:','')

In [27]:
df_train['court_date'] = df_train['court_date'].str.replace('تاريخها: ','')

In [28]:
df_val['court_date'] = df_val['court_date'].str.replace('تاريخها: ','')

In [29]:
df_train['appeal_court'] = df_train['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [30]:
df_val['appeal_court'] = df_val['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [31]:
df_train['region'] = df_train['region'].str.replace('المدينة: ','')

In [32]:
df_val['region'] = df_val['region'].str.replace('المدينة: ','')

In [33]:
df_train['city'] = df_train['city'].str.replace('المدينه: ','')

In [34]:
df_val['city'] = df_val['city'].str.replace('المدينه: ','')

In [35]:
df_train.city.value_counts()

الرياض             148
الدمام              14
المدينة المنورة     12
بريدة                5
Name: city, dtype: int64

In [36]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
# df_train.appeal_text = df_train.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [37]:
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
df_val.judgment_text = df_val.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
# df_val.appeal_text = df_val.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [38]:
df_train

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text
299,القضية رقم ٦٨١٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٦٨١٧,٢٥/١١/١٤٤٢,المحكمة التجارية,٦٨٦,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
33,القضية رقم ٥٨٥٣ لعام ١٤٤٢ هـ,المحكمة التجارية,٥٨٥٣,٢١/٩/١٤٤٢,المحكمة التجارية,٥٢٠٥,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
39,القضية رقم ٦٨١٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٦٨١٧,٢٥/١١/١٤٤٢,المحكمة التجارية,٦٨٦,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
259,القضية رقم ٤٤٣١ لعام ١٤٣٩ هـ,المحكمة التجارية,٤٤٣١,١٤/٤/١٤٤١,المحكمة التجارية,٣٥٨٢,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
297,القضية رقم ٦٤٤٦ لعام ١٤٣٩ هـ,المحكمة التجارية,٦٤٤٦,٣٠/١/١٤٤١,المحكمة التجارية,٣٦٨٣,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,القضية رقم ١٦٥٦٣ لعام ١٤٤٠ هـ,المحكمة التجارية,١٦٥٦٣,١٨/٣/١٤٤٢,المحكمة التجارية,٢٩٨٥,٥/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
46,القضية رقم ٣١٨٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٣١٨٧,٢٣/١٢/١٤٤٢,المحكمة التجارية,٥٩١,٦/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
239,القضية رقم ٧٤٥٧ لعام ١٤٣٩ هـ,المحكمة التجارية,٧٤٥٧,٩/٩/١٤٤٢,المحكمة التجارية,٥٢٥٣,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
308,القضية رقم ١٨٥٩ لعام ١٤٤١ هـ,المحكمة التجارية,١٨٥٩,١٨/١١/١٤٤٢,محكمة الاستئناف,١١٨,٧/٢/١٤٤٣,المنطقة الشرقية,الدمام,الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجا...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN


# Preprocessing

### Remove stopwords

In [39]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stp.stopwords_list()]))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda y: ' '.join([word for word in y.split() if word not in stp.stopwords_list()]))


### Correct words

In [40]:
corr = Corrector()
# df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: corr.spell_correct(x))

### Remove digits

In [41]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))

In [42]:
df_train['judgment_text'] = df_train['judgment_text'].str.replace('\n','')

In [43]:
ArListem = ArabicLightStemmer()
ArListem.light_stem('منجزين')

'منجز'

In [44]:
ISRIStemmer().suf32("منجزين")

'منجز'

In [45]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ArListem.light_stem(x))
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ISRIStemmer().suf32(x))


In [46]:
# corr.spell_correct(df_train['judgment_text'][0])

In [47]:
df_train['judgment_text'][0]

'الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجارية الثانية والعشرون وبناء القضية رقم  لعام  المقامة عبدﷲ ابراهيم المحمود سجل تجاري ضد تغريد بنت محمد صفير هوية وطنية القاضي عمر مبارك الدوسري رئيسا الوقائع تتلخص وقائع الدعوى وبالقدر اللازم لإصدار الحكم وردت المحكمة التجارية لائحة دعوى مقدمة المدعي وكالة ضد المدعى والمثبتة بياناتها ملف القضية جاء لائحة الدعوى اتفق موكلي الجهة المدعى بتأريخ   لتنفيذ أعمال الموقع الخارجي لمشروع القوات البحرية الملكية السعودية الجبيل وتمثل  أعمال المشروع والمتمثلة أعمال السفلتة وغرف التفتيش وعددها  غرفة ومدة العقد  خمسة عشر يوما تنجز الجهة المدعى  الأعمال وعدد  غرف تفتيش عدد  غرفة واستلموا كامل المبلغ وقدره  مئتان واثنان وعشرون الف ريال إتمام المشروع والمستحق الفعلي  مئة وعشرة ألف ريالا أطلب يلزم الجهة المدعى مؤسسة قمة الصحراء للمقاولات العامة بدفع لموكلتي مبلغ وقدره  مئة واثنا عشر ألف ريالا قيدت الأوراق قضية بالرقم المشار بعالية أحيلت الدائرة فباشرت نظرها النحو الموضح بمحاضر الضبط جلسة يوم  حضر وكيل المدعي بوكالة رقم  ذكر المدعي دعواه مقامة شركة بيت الجزيرة

# remove english words

In [48]:
df_train.judgment_text = df_train.judgment_text.apply(lambda x: re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', x) )

# TF-IDF

In [49]:
cv_tfidf = TfidfVectorizer(min_df=2, max_df=20)
x = cv_tfidf.fit_transform(df_train['judgment_text']).toarray()

In [50]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
dt_tfidf = pd.DataFrame(x,columns=cv_tfidf.get_feature_names())
dt_tfidf


,آب,آت,آثارها,آجل,آخرين,آر,آكد,آلاف,آلة,آلت,آلي,آلية,آمنوا,آن,آنف,...,يوضح,يوفر,يوفق,يوفقكم,يوفي,يوقع,يومنا,يوميا,يومية,يومين,يونيه,يونيو,يوهمني,يي,ييثبت
0,0.0,0.0,0.0,0.115271,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.115271,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.045473,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.023678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Cosine similarity


In [52]:
# df.to_csv('new.csv',index=False)

In [72]:
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity


cosine_similarity(dt_tfidf.iloc[:,1:2],dt_tfidf.iloc[:,2:3])
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))


In [80]:
cosine(dt_tfidf.iloc[20],dt_tfidf.iloc[51])

1.0000000000000002

In [84]:
x = range(1,197)
y = range(1,197)
for i in x:
    for j in y:
        print(cosine(dt_tfidf.iloc[i],dt_tfidf.iloc[j]))
        print('i= ', i)
        print('j= ', j)
        if cosine(dt_tfidf.iloc[i],dt_tfidf.iloc[j]) > 0.5 and i != j:
            print("---------",cosine(dt_tfidf.iloc[i],dt_tfidf.iloc[j]))
        

1.0
i=  1
j=  1
0.0573774769574995
i=  1
j=  2
0.020096947864038938
i=  1
j=  3
0.008673515872828587
i=  1
j=  4
0.006852093632321316
i=  1
j=  5
0.008220609946818587
i=  1
j=  6
0.02751759227168046
i=  1
j=  7
0.0017362137273743152
i=  1
j=  8
0.019136064830168706
i=  1
j=  9
0.00799062148360443
i=  1
j=  10
0.03148643023100209
i=  1
j=  11
0.02751997773015902
i=  1
j=  12
0.03069660065171707
i=  1
j=  13
0.00775570767638696
i=  1
j=  14
0.009485943639103931
i=  1
j=  15
0.003662177416789803
i=  1
j=  16
0.01396467532308022
i=  1
j=  17
0.05010909456519788
i=  1
j=  18
0.01859590422080808
i=  1
j=  19
0.0185833040480581
i=  1
j=  20
0.008104975099833971
i=  1
j=  21
0.0
i=  1
j=  22
0.008751718696421388
i=  1
j=  23
0.0
i=  1
j=  24
0.00923187067839965
i=  1
j=  25
0.07570718459701914
i=  1
j=  26
0.008286855283639696
i=  1
j=  27
0.02985749498545847
i=  1
j=  28
0.0024144755175073956
i=  1
j=  29
0.04642357967369461
i=  1
j=  30
0.012652787663622333
i=  1
j=  31
0.01661669873665771
i

0.014752106215891373
i=  2
j=  182
0.0
i=  2
j=  183
0.005265417077092801
i=  2
j=  184
0.0
i=  2
j=  185
0.014838899191236748
i=  2
j=  186
0.008622432179074949
i=  2
j=  187
0.01847636604281475
i=  2
j=  188
0.0
i=  2
j=  189
0.0
i=  2
j=  190
0.011313323626432508
i=  2
j=  191
0.005818023897783582
i=  2
j=  192
0.00565505829153073
i=  2
j=  193
0.11888907873986478
i=  2
j=  194
0.0065482054625438935
i=  2
j=  195
0.008594938693945174
i=  2
j=  196
0.020096947864038938
i=  3
j=  1
0.006591463908198878
i=  3
j=  2
1.0
i=  3
j=  3
0.0189494200169539
i=  3
j=  4
0.04021339174382522
i=  3
j=  5
0.022602117918779398
i=  3
j=  6
0.013497598377144142
i=  3
j=  7
0.004514435183763643
i=  3
j=  8
0.010128785634706997
i=  3
j=  9
0.013552044692303929
i=  3
j=  10
0.0073454479165063315
i=  3
j=  11
0.0540914828597709
i=  3
j=  12
0.06290521056231932
i=  3
j=  13
0.02648645768246459
i=  3
j=  14
0.015289358463878596
i=  3
j=  15
0.022099036085098885
i=  3
j=  16
0.014699253819327263
i=  3
j=  17

0.009054421851416783
i=  4
j=  124
0.01936163234710548
i=  4
j=  125
0.008668168444406002
i=  4
j=  126
0.0018646585527176244
i=  4
j=  127
0.026285734885052377
i=  4
j=  128
0.015285692219783042
i=  4
j=  129
0.11524682952021047
i=  4
j=  130
0.015083778081505302
i=  4
j=  131
0.04239349853696539
i=  4
j=  132
0.001634481687860471
i=  4
j=  133
0.010402274562771447
i=  4
j=  134
0.010488092460904187
i=  4
j=  135
0.021826112489590686
i=  4
j=  136
0.060700034373456675
i=  4
j=  137
0.02409286906995005
i=  4
j=  138
0.002029437215033761
i=  4
j=  139
0.009616745701220751
i=  4
j=  140
1.0
i=  4
j=  141
--------- 1.0
0.00436294361798121
i=  4
j=  142
0.017945219344220387
i=  4
j=  143
0.00436294361798121
i=  4
j=  144
0.006432741413248039
i=  4
j=  145
0.09029736605968801
i=  4
j=  146
0.02697300250970279
i=  4
j=  147
0.01271553430584765
i=  4
j=  148
0.023496140927759836
i=  4
j=  149
0.004529690920716906
i=  4
j=  150
0.002029437215033761
i=  4
j=  151
0.0022838536289780343
i=  4
j= 

0.03681397732891639
i=  6
j=  113
0.004641049463119309
i=  6
j=  114
0.005749531572351561
i=  6
j=  115
0.023327511204621613
i=  6
j=  116
0.03158446427493148
i=  6
j=  117
0.014154098312275275
i=  6
j=  118
0.018313410157746554
i=  6
j=  119
0.013567662604366515
i=  6
j=  120
0.008249520091891258
i=  6
j=  121
0.0
i=  6
j=  122
0.004812674777686281
i=  6
j=  123
0.14209238922566994
i=  6
j=  124
0.01322307326110569
i=  6
j=  125
0.0036545820282838123
i=  6
j=  126
0.0426296204276154
i=  6
j=  127
0.00549032682329082
i=  6
j=  128
0.00612708152422532
i=  6
j=  129
0.019934674997837687
i=  6
j=  130
0.010993663281658576
i=  6
j=  131
0.007555463037839752
i=  6
j=  132
0.020315683888756746
i=  6
j=  133
0.00972951319312592
i=  6
j=  134
0.06459535872912602
i=  6
j=  135
0.0354103002254051
i=  6
j=  136
0.02597627746587465
i=  6
j=  137
0.01977468631681868
i=  6
j=  138
0.015739074755651852
i=  6
j=  139
0.008772354639893048
i=  6
j=  140
0.024246565373723522
i=  6
j=  141
0.0912060229610

0.09177589713039885
i=  8
j=  69
0.007857910752815993
i=  8
j=  70
0.004174914572398304
i=  8
j=  71
0.025820344515583186
i=  8
j=  72
0.0
i=  8
j=  73
0.014417177966441224
i=  8
j=  74
0.02870674183555116
i=  8
j=  75
0.002734750841102295
i=  8
j=  76
0.0012048287911037483
i=  8
j=  77
0.0
i=  8
j=  78
0.00737281720075956
i=  8
j=  79
0.0431889987340507
i=  8
j=  80
0.004175857373904926
i=  8
j=  81
0.01894298480275256
i=  8
j=  82
0.011686482336772552
i=  8
j=  83
0.00852311949930123
i=  8
j=  84
0.0
i=  8
j=  85
0.00852311949930123
i=  8
j=  86
0.010416297514647933
i=  8
j=  87
0.004578354789610458
i=  8
j=  88
0.004634158287108644
i=  8
j=  89
0.0
i=  8
j=  90
0.0023905940016253876
i=  8
j=  91
0.012359641888744164
i=  8
j=  92
0.005058606344897449
i=  8
j=  93
0.005182433676744758
i=  8
j=  94
0.007857910752815993
i=  8
j=  95
0.018624827318917396
i=  8
j=  96
0.013030225316722528
i=  8
j=  97
0.0
i=  8
j=  98
0.009993357447263465
i=  8
j=  99
0.0
i=  8
j=  100
0.10022606080223305

...

0.011826180757983428
i=  188
j=  130
0.014474274640589476
i=  188
j=  131
0.008034087521813614
i=  188
j=  132
0.0542485055488109
i=  188
j=  133
0.0023739734354913966
i=  188
j=  134
0.04011108737163283
i=  188
j=  135
0.12776076341807627
i=  188
j=  136
0.008188056674325718
i=  188
j=  137
0.005660270256183706
i=  188
j=  138
0.029061852695312418
i=  188
j=  139
0.042397114377055664
i=  188
j=  140
0.0070788691850896274
i=  188
j=  141
0.013515343490638012
i=  188
j=  142
0.0
i=  188
j=  143
0.013515343490638012
i=  188
j=  144
0.0
i=  188
j=  145
0.03576893002055796
i=  188
j=  146
0.0067878279608260764
i=  188
j=  147
0.044755942250846885
i=  188
j=  148
0.03160405673075926
i=  188
j=  149
0.0
i=  188
j=  150
0.029061852695312418
i=  188
j=  151
0.011824328979071432
i=  188
j=  152
0.07407095947354446
i=  188
j=  153
0.013870357401973583
i=  188
j=  154
0.0
i=  188
j=  155
0.016105216224847955
i=  188
j=  156
0.028323641960247982
i=  188
j=  157
0.005660270256183706
i=  188
j=  158

0.02369589810166249
i=  190
j=  87
0.020140859804188324
i=  190
j=  88
0.021350067246337017
i=  190
j=  89
0.006000083918353751
i=  190
j=  90
0.10977451849366338
i=  190
j=  91
0.019115295611346318
i=  190
j=  92
0.0
i=  190
j=  93
0.0058752874968414515
i=  190
j=  94
0.031898208907431044
i=  190
j=  95
0.011762461796229171
i=  190
j=  96
0.04050430214296736
i=  190
j=  97
0.00425133508815161
i=  190
j=  98
0.0
i=  190
j=  99
0.0
i=  190
j=  100
0.0031184248805219503
i=  190
j=  101
0.011762461796229171
i=  190
j=  102
0.008590776857578454
i=  190
j=  103
0.006035964260961851
i=  190
j=  104
0.027945009289742345
i=  190
j=  105
0.002077816254266389
i=  190
j=  106
0.013872583816746279
i=  190
j=  107
0.0058752874968414515
i=  190
j=  108
0.010184243538866943
i=  190
j=  109
0.028826178112366662
i=  190
j=  110
0.018508802029367666
i=  190
j=  111
0.02385466481788392
i=  190
j=  112
0.01134596822968781
i=  190
j=  113
0.0136443528823028
i=  190
j=  114
0.0
i=  190
j=  115
0.0
i=  190
j

0.005650218675348263
i=  192
j=  37
0.010871569372366253
i=  192
j=  38
0.08272864019119405
i=  192
j=  39
0.015058555519126056
i=  192
j=  40
0.0074987559888498225
i=  192
j=  41
0.005793517628516006
i=  192
j=  42
0.004020063087083737
i=  192
j=  43
0.035506323673755134
i=  192
j=  44
0.022650626749852302
i=  192
j=  45
0.005818023897783582
i=  192
j=  46
0.01758889141768968
i=  192
j=  47
0.00846544079204539
i=  192
j=  48
0.002998930190768816
i=  192
j=  49
0.007158361468795564
i=  192
j=  50
0.032522798964876067
i=  192
j=  51
0.07287996226135389
i=  192
j=  52
0.01295135437057271
i=  192
j=  53
0.00933710006002696
i=  192
j=  54
0.0
i=  192
j=  55
0.016891321831725436
i=  192
j=  56
0.02967001563499114
i=  192
j=  57
0.015083110437776468
i=  192
j=  58
0.03948168602832563
i=  192
j=  59
0.00846544079204539
i=  192
j=  60
0.020254162655935833
i=  192
j=  61
0.003812769210568587
i=  192
j=  62
0.03333864618111705
i=  192
j=  63
0.03587464895041751
i=  192
j=  64
0.05009066717091814

0.0023334188308646218
i=  194
j=  3
0.006781155764386946
i=  194
j=  4
0.014777383047001916
i=  194
j=  5
0.0042288801766188445
i=  194
j=  6
0.009316077901269353
i=  194
j=  7
0.0021743862179873052
i=  194
j=  8
0.006018557164570356
i=  194
j=  9
0.007539928741412466
i=  194
j=  10
0.011428653445997613
i=  194
j=  11
0.01968530319806894
i=  194
j=  12
0.02563901002520203
i=  194
j=  13
0.0036644390207856386
i=  194
j=  14
0.004634141712786123
i=  194
j=  15
0.019824327211186796
i=  194
j=  16
0.03079614650228798
i=  194
j=  17
0.0313181857912664
i=  194
j=  18
0.022860938415950976
i=  194
j=  19
0.009236375648715846
i=  194
j=  20
0.005271514514850268
i=  194
j=  21
0.018676892116106784
i=  194
j=  22
0.01615924448409606
i=  194
j=  23
0.06576893679553666
i=  194
j=  24
0.004809718194410183
i=  194
j=  25
0.012791166586187994
i=  194
j=  26
0.0073711380686203335
i=  194
j=  27
0.021722138278559046
i=  194
j=  28
0.020063284171223886
i=  194
j=  29
0.002049338129141364
i=  194
j=  30
0

0.008018498724210338
i=  195
j=  147
0.01395798301513844
i=  195
j=  148
0.033695636480150344
i=  195
j=  149
0.05087789528905522
i=  195
j=  150
0.013194790667224232
i=  195
j=  151
0.013032046461020956
i=  195
j=  152
0.032546150052549995
i=  195
j=  153
0.03697165897793984
i=  195
j=  154
0.009166147977269317
i=  195
j=  155
0.02461862240931901
i=  195
j=  156
0.01603953875965514
i=  195
j=  157
0.057161606590940044
i=  195
j=  158
0.0083643524555632
i=  195
j=  159
0.008566100810952531
i=  195
j=  160
0.01621772769975665
i=  195
j=  161
0.013614482062346953
i=  195
j=  162
0.015681816834565033
i=  195
j=  163
0.014485209199763762
i=  195
j=  164
0.014485209199763762
i=  195
j=  165
0.00415806916979207
i=  195
j=  166
0.012992613391338835
i=  195
j=  167
0.021001069380604455
i=  195
j=  168
0.08754946887158954
i=  195
j=  169
0.05425903051393628
i=  195
j=  170
0.017606810341495378
i=  195
j=  171
0.01928552636987486
i=  195
j=  172
0.003939385177041311
i=  195
j=  173
0.02438408914

In [82]:
df_train.judgment_text.iloc[20]

'الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجارية الثانية والعشرون وبناء القضية رقم  لعام  المقامة يحيى وليد يحيى المدني سجل تجاري ضد كوكب الاعمار للمقاولات لصاحبها هشام فهمي المتولي الشيخ القاضي عمر مبارك الدوسري رئيسا الوقائع تتلخص وقائع الدعوى وبالقدر اللازم لإصدار الحكم وردت المحكمة التجارية لائحة دعوى مقدمة المدعي وكالة ضد المدعى والمثبتة بياناتها ملف القضية جاء لائحة الدعوى اطالب المؤسسة المدعى مؤسسة كوكب الاعمار للمقاولات مبلغ وقدره  ريال سعودي مقابل قامت المدعية مؤسسة سنمار للتجارة بتوريد مواد كهربائية للمدعي بمبلغ وقدرة  ريال وقامت المدعية بتوجيه كتاب بتاريخ  للمدعي مؤسسة كوكب الاعمار بمطابقة الرصيد والختم والمصادقة والاقرار قامت المدعى بتحرير عدد  شيكات لصالح المدعية عرضهما البنك تبين انه يوجد رصيد كافي لصرف الشيكات وقيمة الشيكات مبلغ وقدره  ريال سعودي كالتالي  شيك رقم  بقيمة  ريال سعودي  شيك رقم  بقيمة  ريال سعودي علمن ان قيمة المطالبة مترتبة بذمة المدعى قرابة السنتين تمت المطالبة مرارا وتكرار الا ان المدعى امتنعت السداد ارجو الزام المدعى بدفع المبلغ المطلوب كامل وقدره  ريا

In [83]:
df_train.judgment_text.iloc[51]

'الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجارية الثانية والعشرون وبناء القضية رقم لعام المقامة يحيى وليد يحيى المدني سجل تجاري ضد كوكب الاعمار للمقاولات لصاحبها هشام فهمي المتولي الشيخ القاضي عمر مبارك الدوسري رئيسا الوقائع تتلخص وقائع الدعوى وبالقدر اللازم لإصدار الحكم وردت المحكمة التجارية لائحة دعوى مقدمة المدعي وكالة ضد المدعى والمثبتة بياناتها ملف القضية جاء لائحة الدعوى اطالب المؤسسة المدعى مؤسسة كوكب الاعمار للمقاولات مبلغ وقدره ريال سعودي مقابل قامت المدعية مؤسسة سنمار للتجارة بتوريد مواد كهربائية للمدعي بمبلغ وقدرة ريال وقامت المدعية بتوجيه كتاب بتاريخ للمدعي مؤسسة كوكب الاعمار بمطابقة الرصيد والختم والمصادقة والاقرار قامت المدعى بتحرير عدد شيكات لصالح المدعية عرضهما البنك تبين انه يوجد رصيد كافي لصرف الشيكات وقيمة الشيكات مبلغ وقدره ريال سعودي كالتالي شيك رقم بقيمة ريال سعودي شيك رقم بقيمة ريال سعودي علمن ان قيمة المطالبة مترتبة بذمة المدعى قرابة السنتين تمت المطالبة مرارا وتكرار الا ان المدعى امتنعت السداد ارجو الزام المدعى بدفع المبلغ المطلوب كامل وقدره ريال سعودي الزامه